<a href="https://colab.research.google.com/github/nacamacho/MedicBot/blob/main/Med_360__Redes_Neuronales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
# Importar las librerías necesarias
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from google.colab import auth
import gspread
from google.auth import default
import numpy as np

# Autenticar y autorizar con Google Colab
auth.authenticate_user()

# Autorizar gspread
creds, _ = default()
gc = gspread.authorize(creds)

# Abrir la hoja de cálculo por nombre
spreadsheet = gc.open("Encuesta Med 360 (respuestas)")
sheet = spreadsheet.sheet1

# Convertir la hoja de cálculo a un DataFrame de pandas
data = pd.DataFrame(sheet.get_all_records())

# Selección de las columnas relevantes
features = data[['Edad', 'Ciudad', 'Que EPS pertenece', 'Ha utilizado el chatbot para agendar citas médicas',
                 'Qué tan satisfecho está con la interfaz y la facilidad de uso del chatbot?']]
target = data['¿Recomendaría el uso del chatbot para agendar citas médicas a otras personas?']

# Codificación de variables categóricas
label_encoders = {}
for column in features.select_dtypes(include=['object']).columns:
    label_encoders[column] = LabelEncoder()
    # Ajustar el LabelEncoder con todos los datos únicos posibles
    label_encoders[column].fit(features[column])

# Transformar las características
for column in features.select_dtypes(include=['object']).columns:
    features[column] = label_encoders[column].transform(features[column])

# Codificación del target
target_encoder = LabelEncoder()
target = target_encoder.fit_transform(target)

# Normalización de características
scaler = StandardScaler()
features = scaler.fit_transform(features)

# División de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Construcción del modelo de red neuronal
model = Sequential()
model.add(Dense(16, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compilación del modelo
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Entrenamiento del modelo
model.fit(X_train, y_train, epochs=50, batch_size=10, validation_split=0.2)

# Evaluación del modelo
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Loss: {loss}, Accuracy: {accuracy}')

# Hacer predicciones sobre el conjunto de prueba
y_pred = model.predict(X_test)

# Convertir las predicciones a binario (0 o 1)
y_pred_classes = (y_pred > 0.5).astype("int32")

# Comparar las predicciones con los valores reales
print("Classification Report:")
print(classification_report(y_test, y_pred_classes))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_classes))

# Crear un chatbot para interactuar con el modelo entrenado
def interact_with_model():
    print("Escribe 'adiós', 'salir' o 'exit' para terminar la conversación.")
    while True:
        # Obtener entrada del usuario
        edad = input("Tú (Edad): ")
        ciudad = input("Tú (Ciudad): ")
        eps = input("Tú (EPS): ")
        ha_usado_chatbot = input("Tú (Ha utilizado el chatbot para agendar citas médicas - Si/No): ")
        satisfaccion = input("Tú (Qué tan satisfecho está con la interfaz y la facilidad de uso del chatbot?): ")

        # Salir del bucle si el usuario escribe 'adiós'
        if edad.lower() in ["adiós", "salir", "exit"]:
            print("Chatbot: ¡Adiós!")
            break

        # Crear un DataFrame con los datos del usuario
        nuevos_datos = pd.DataFrame({
            'Edad': [edad],
            'Ciudad': [ciudad],
            'Que EPS pertenece': [eps],
            'Ha utilizado el chatbot para agendar citas médicas': [ha_usado_chatbot],
            'Qué tan satisfecho está con la interfaz y la facilidad de uso del chatbot?': [satisfaccion]
        })

        # Aplicar el mismo preprocesamiento que a los datos de entrenamiento
        for column in nuevos_datos.select_dtypes(include=['object']).columns:
            if column in label_encoders:
                # Manejar etiquetas no vistas
                le = label_encoders[column]
                le_classes = list(le.classes_)
                new_classes = nuevos_datos[column].unique()
                for new_class in new_classes:
                    if new_class not in le_classes:
                        le_classes.append(new_class)
                le.classes_ = np.array(le_classes)
                nuevos_datos[column] = le.transform(nuevos_datos[column])

        nuevos_datos = scaler.transform(nuevos_datos)

        # Hacer predicciones sobre los nuevos datos
        prediccion_nueva = model.predict(nuevos_datos)
        prediccion_clase = (prediccion_nueva > 0.5).astype("int32")
        recomendacion = target_encoder.inverse_transform(prediccion_clase)[0]

        # Imprimir la predicción del chatbot
        print(f"Chatbot: Según tu perfil, {'sí' if prediccion_clase[0][0] == 1 else 'no'} recomendarías el uso del chatbot para agendar citas médicas a otras personas.")

# Llamar a la función para iniciar la interacción
interact_with_model()


<ipython-input-10-b0c277a33860>:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features[column] = label_encoders[column].transform(features[column])
<ipython-input-10-b0c277a33860>:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features[column] = label_encoders[column].transform(features[column])
<ipython-input-10-b0c277a33860>:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the 

Epoch 1/50
2/2 [==============================] - 3s 1s/step - loss: 0.6682 - accuracy: 0.6316 - val_loss: 0.7027 - val_accuracy: 0.4000
Epoch 2/50
2/2 [==============================] - 0s 130ms/step - loss: 0.6629 - accuracy: 0.6316 - val_loss: 0.6964 - val_accuracy: 0.6000
Epoch 3/50
2/2 [==============================] - 0s 247ms/step - loss: 0.6585 - accuracy: 0.6316 - val_loss: 0.6908 - val_accuracy: 0.6000
Epoch 4/50
2/2 [==============================] - 0s 183ms/step - loss: 0.6542 - accuracy: 0.6316 - val_loss: 0.6846 - val_accuracy: 0.8000
Epoch 5/50
2/2 [==============================] - 0s 137ms/step - loss: 0.6506 - accuracy: 0.6316 - val_loss: 0.6784 - val_accuracy: 0.8000
Epoch 6/50
2/2 [==============================] - 0s 105ms/step - loss: 0.6459 - accuracy: 0.6842 - val_loss: 0.6730 - val_accuracy: 0.8000
Epoch 7/50
2/2 [==============================] - 0s 164ms/step - loss: 0.6421 - accuracy: 0.6842 - val_loss: 0.6669 - val_accuracy: 0.8000
Epoch 8/50
2/2 [=======

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:155: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Chatbot: Según tu perfil, no recomendarías el uso del chatbot para agendar citas médicas a otras personas.
Tú (Edad): adiós
Tú (Ciudad): bogota
Tú (EPS): compensar
Tú (Ha utilizado el chatbot para agendar citas médicas - Si/No): no
Tú (Qué tan satisfecho está con la interfaz y la facilidad de uso del chatbot?): 0
Chatbot: ¡Adiós!
